In [608]:
from langchain_community.llms import Ollama
import pprint
import yfinance

AGENT_MODEL_NAME = "arcee-ai/arcee-agent"

func_prompt = """You have four primary functions: checking the last price of a specified stock, finding the name of a company CEO, finding what a company does, and answering specific questions about a company. Use the appropriate function based on the user's query.

### Functions:

1. **get_stock_price(company_name: str, stock_symbol: str) -> str**
    - This function returns the last close price of a specified stock.
    - Input: A company name (e.g., Mc Donalds), which you must convert to a stock symbol (e.g., MCD).
    - Output: A string containing the last close price of the specified stock (e.g., "The last closing price of Mc Donalds (MCD) is $250.00").

2. **get_ceo_name(company_name: str, stock_symbol: str) -> str**
- This function returns the name of the CEO of a specified company.
- Input: A company name (e.g., Mc Donalds), which you must convert to a stock symbol (e.g., MCD).
- Output: A string containing the name of the CEO of the specified company (e.g., "The CEO of Mc Donalds is John Doe").

3. **get_company_summary(company_name: str, stock_symbol: str) -> str**
- This function returns a summary describing the business activities of a specified company.
- Input: A company name (e.g., Mc Donalds), which you must convert to a stock symbol (e.g., MCD).
- Output: A string containing a detailed summary of the specified company's business activities.

4. **answer_general_question(question: str) -> str**
    - This function answers questions in general.
    - Input: a user question.
    - Output: your best answer to the user question.
   
### Instructions:

- If the user asks a question related to the price of a stock, use the `get_stock_price` function.
- If the user asks a question related to the CEO of a company, use the `get_ceo_name` function.
- If the user asks a general question about a company's activities, use the `get_company_summary` function.
- If the user asks any other question, use the `answer_general_question` function. Only return the result of the function call, not your internal reasoning.
- Only return the result of the function call.

### User Query:\n"""

agent = Ollama(
    model=AGENT_MODEL_NAME,
    system="""You are a helpful chat assistant.""",
    top_p=0.7,
)  # assuming you have run  `ollama pull arcee-ai/arcee-agent`


def function_call(input, model=agent, func_prompt=func_prompt):
    payload = f"""{func_prompt}{input}\n"""
    ans = model.invoke(payload)
    return(ans)


In [616]:
def get_stock_price(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    price = stock.history(period="1d")["Close"].values[0]
    return f"The last closing price of {company_name} ({stock_symbol}) was ${price:.2f}."

def get_ceo_name(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    info = stock.info
    ceo = info['companyOfficers'][0]['name']
    return f"The CEO of {company_name} is {ceo}. The full job title is {info['companyOfficers'][0]['title']}."

def get_company_summary(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    summary = stock.info['longBusinessSummary']
    return f"{company_name} ({stock_symbol}) is a company that is involved in {summary}."

def answer_general_question(question):
    ans = agent.invoke(f"""{question}\n""")
    return(ans)

def llm_pack(input):
    try:
        func = function_call(input).strip()
        print(func)
        code = f"result = {func}"
        local_vars = {}
        exec(code, globals(), local_vars)
        ans = local_vars.get('result').strip()
        return ans
    except Exception as e:
        return

In [617]:
response = llm_pack("What the stock price for Caterpillar?")
print(response)

get_stock_price('Caterpillar', 'CAT')
The last closing price of Caterpillar (CAT) was $320.87.


In [618]:
response = llm_pack("Who runs Caterpillar?")
print(response)

get_ceo_name("Caterpillar", "CAT")
The CEO of Caterpillar is Mr. D. James Umpleby III. The full job title is Chairman of the Board & CEO.


In [619]:
response = llm_pack("What does Caterpillar build?")
pprint.pprint(response)

get_company_summary("Caterpillar", "CAT")
('Caterpillar (CAT) is a company that is involved in Caterpillar Inc. '
 'manufactures and sells construction and mining equipment, off-highway diesel '
 'and natural gas engines, industrial gas turbines, and diesel-electric '
 'locomotives in worldwide. Its Construction Industries segment offers asphalt '
 'pavers, compactors, road reclaimers, forestry machines, cold planers, '
 'material handlers, track-type tractors, excavators, telehandlers, motor '
 'graders, and pipelayers; compact track, wheel, track-type, backhoe, and skid '
 "steer loaders; and related parts and tools. The company's Resource "
 'Industries segment provides electric rope and hydraulic shovels, draglines, '
 'rotary drills, hard rock vehicles, tractors, mining trucks, wheel loaders, '
 'off-highway and articulated trucks, wheel tractor scrapers and dozers, fleet '
 'management products, landfill and soil compactors, machinery components, '
 'autonomous ready vehicles and

In [620]:
response = llm_pack(f"Who are the main competitors of Caterpillar?")
print(response)

answer_general_question("Who are the main competitors of Caterpillar?")
Caterpillar, Inc., is a global leader in construction and mining equipment, diesel and natural gas engines, industrial gas turbines, and diesel-electric locomotives. Some of its main competitors in various segments include:

1. Komatsu Ltd: A Japanese multinational corporation that specializes in the production of construction, mining, forestry, and earthmoving equipment.
2. Deere & Company (John Deere): An American corporation known for manufacturing agricultural, forestry, and construction machinery, as well as engines and lawn care products.
3. Volvo Group: A Swedish multinational corporation that produces trucks, buses, construction equipment, marine and industrial engines, as well as power generation systems.
4. Hitachi Construction Machinery Co., Ltd.: A Japanese company that manufactures a wide range of construction machinery, including excavators, loaders, and compactors.
5. Doosan Heavy Industries & Constr